_inserts one my fav kdot lyric_

**Kendrick Lamar**

Importing necessary libraries

In [3]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

Establishing a Connection with the Spotify API

In [4]:
current_dir = os.path.dirname(os.path.abspath("Kendrick Lamar"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

# Read the config file
config.read(config_path)
            
client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

Function to monitor duration of scrape

In [5]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

Artist Profile harvesting

In [6]:
from spotipy.oauth2 import SpotifyClientCredentials

# Get credentials from the config file
client_id= '31d85b3f430e45caba7b97174cb75137'
client_secret = '5c6b719bcd8743b0ad1f8cd217ad1e1c'
client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [7]:
def collect_spotify_data():
    start_time = timer()
    data = []  # Using a list of dictionaries to collect data

    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist: Kendrick Lamar', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                # Get audio features for track
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    # Handle case where audio features are not available
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                # Get featured artists
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5)  # Retry after a short delay
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break  # Break the loop if there's a persistent error

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print("Elapsed time:", timekeeper(elapsed_time))
    return df_raw

# Collecting data from Spotify API 
df_spotify = collect_spotify_data()

Elapsed time: 3 minutes, 32 seconds


In [36]:
df_spotify

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Nipsey Hussle,Dedication,1zRTFvd3dpzlr75zVTKf6O,Visionary Artists,2qYP3N8Q23coynyjw18rcg,2024-03-12,0,True,0.513,245813,...,-5.448,0,0.4320,0.431000,0.000000,0.2260,0.170,94.527,4,[Kendrick Lamar]
1,Kendrick Lamar,Not Like Us,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,2024-05-04,90,True,0.898,274192,...,-7.001,1,0.0776,0.010700,0.000000,0.1410,0.214,101.061,4,[]
2,Various Artists,Alan Anderson - “Levitate” by Kendrick Lamar,4u3pOE7VcP0LX7HhugS0yc,LA Clippers - Cali Fire,5xIwaxMrnDHJwVWwXhiu4W,2016-10-31,0,False,0.000,6000,...,-7.246,1,0.0000,0.982000,0.000000,0.0000,0.000,0.000,0,[]
3,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,78,True,0.716,386907,...,-7.355,1,0.1220,0.070300,0.000000,0.2240,0.344,71.994,4,[Jay Rock]
4,Kendrick Lamar,DNA.,6HZILIRieu8S0iqY8kIKhj,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,79,True,0.638,185947,...,-6.664,1,0.3570,0.004540,0.000000,0.0842,0.422,139.913,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
797,Grant Richards,All The Stars (Tribute to Kendrick Lamar & SZA),3hHdXXndhSq979D5omW8Dq,All The Stars (Tribute to Kendrick Lamar & SZA),7que97Xsxcj5jsDfOd6ENB,2018-01-18,0,False,0.694,230322,...,-7.668,1,0.0704,0.022100,0.279000,0.1100,0.417,96.682,4,[]
798,New Tribute Kings,The Greatest (Originally Performed By Sia Feat...,0RqWmSFYHYNigOMM1Z8knN,It's The Hits 2016! Vol. 11,3L8T1uqT25JkjoYQksVKK7,2016-10-05,0,False,0.738,214104,...,-14.382,1,0.0662,0.002430,0.000060,0.0581,0.832,96.000,4,[]
799,Kendrick Miros,Electric Bass Domination,5kO3OVkjODSe3BdETQZYbB,After Party,4vRXTAEoeQPDJ6Jm38vGZm,2023-06-09,1,False,0.802,135896,...,-9.405,0,0.0927,0.000202,0.924000,0.0540,0.558,128.993,4,[]
800,Kendolff,Desert Camel (7 beer Bitch),0hPdYkfikRh8ijDMKoR8mI,Desert Camel (7 beer Bitch),0O38Bn3CjuVj3BhH6fvoFV,2021-04-27,0,True,0.643,338122,...,-9.342,1,0.0283,0.202000,0.004650,0.3660,0.507,96.080,4,[]


Filtering the artist name column to contain only 'Kendrick Lamar'

In [10]:
kdot = df_spotify[df_spotify['artist_name']=='Kendrick Lamar']
kdot.head()

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
1,Kendrick Lamar,Not Like Us,6AI3ezQ4o3HUoP6Dhudph3,Not Like Us,5JjnoGJyOxfSZUZtk2rRwZ,2024-05-04,90,True,0.898,274192,...,-7.001,1,0.0776,0.01070,0.0,0.1410,0.214,101.061,4,[]
3,Kendrick Lamar,Money Trees,2HbKqm4o0w5wEeEFXm2sD4,"good kid, m.A.A.d city",6PBZN8cbwkqm1ERj2BGXJ1,2012,78,True,0.716,386907,...,-7.355,1,0.1220,0.07030,0.0,0.2240,0.344,71.994,4,[Jay Rock]
4,Kendrick Lamar,DNA.,6HZILIRieu8S0iqY8kIKhj,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,79,True,0.638,185947,...,-6.664,1,0.3570,0.00454,0.0,0.0842,0.422,139.913,4,[]
5,Kendrick Lamar,Alright,3iVcZ5G6tvkXZkZKlMpIUs,To Pimp A Butterfly,7ycBtnsMtyVbbwTfJwRjSP,2015-03-16,79,True,0.796,219333,...,-5.974,1,0.2380,0.07420,0.0,0.0827,0.558,110.034,4,[]
6,Kendrick Lamar,LOVE. FEAT. ZACARI.,6PGoSes0D9eUDeeAafB2As,DAMN.,4eLPsYPBmXABThSJ821sqY,2017-04-14,81,True,0.800,213400,...,-7.343,1,0.0924,0.26400,0.0,0.1530,0.779,126.058,4,[Zacari]


In [32]:
kdot['album_name'].unique()

array(['Not Like Us', 'good kid, m.A.A.d city', 'DAMN.',
       'To Pimp A Butterfly', 'meet the grahams', 'Section.80',
       'Mr. Morale & The Big Steppers', 'good kid, m.A.A.d city (Deluxe)',
       'i', 'untitled unmastered.', 'Alright', 'Overly Dedicated',
       'DAMN. COLLECTORS EDITION.', 'untitled 07 | levitate', 'Top 30 US',
       'The Heart Part 5', 'Gangsta Bangaz Vol. 1', 'Rainy Day R&B',
       'Rap Gods', 'We Run L.A.', 'Hip Hop Hit List (Vol. 3)',
       'Beaches & Bridges - La 2 da Bay, Vol. 5',
       'The Hate U Give (Original Motion Picture Soundtrack)',
       'West Coast Hip-Hop', 'LoFi Chillhop', 'Best Of 2017',
       'Hip Hop Hit List - Vol. IV', 'Spring Party 2018',
       '2018 Award Nominees'], dtype=object)

Categorizing the extract according to what type of release

In [51]:
studio_albums = ['Section.80',
                 'good kid, m.A.A.d city',
                 'To Pimp A Butterfly',                 
                 'DAMN.',
                 'DAMN. COLLECTORS EDITION.', 
                 'Mr. Morale & The Big Steppers']

compilation_album = ['untitled unmastered.']

deluxe_album = ['good kid, m.A.A.d city (Deluxe)']

mixtape = ['Overly Dedicated']

singles = kdot['album_name'] == kdot['track_name']
album_condition = kdot['album_name'].isin(studio_albums)
compilations = kdot['album_name'].isin(compilation_album)
deluxe = kdot['album_name'].isin(deluxe_album)
mixtape_mapping = kdot['album_name'].isin(mixtape)

conditions = [singles, album_condition, compilations, deluxe, mixtape_mapping]

choices = ['Single', 'Studio Album', 'Compilation', 'Deluxe Album', 'Mixtape']

kdot['type'] = np.select(conditions, choices, default= '')

kdot[['artist_name', 'track_name', 'album_name', 'type']].head()

C:\Users\HP\AppData\Local\Temp\ipykernel_13804\726595935.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kdot['type'] = np.select(conditions, choices, default= '')


,artist_name,track_name,album_name,type
1,Kendrick Lamar,Not Like Us,Not Like Us,Single
3,Kendrick Lamar,Money Trees,"good kid, m.A.A.d city",Studio Album
4,Kendrick Lamar,DNA.,DAMN.,Studio Album
5,Kendrick Lamar,Alright,To Pimp A Butterfly,Studio Album
6,Kendrick Lamar,LOVE. FEAT. ZACARI.,DAMN.,Studio Album


scraping a spotify playlist that contains all Kendrick's features 

In [41]:
# start time
start_time = timer()

# Get playlist URI
playlist_uri = "https://open.spotify.com/playlist/5Z7uz9iDMfINUocPZbgHRG?si=8JTn4Ae-SEy6SCEYSjIm6w"

# Get playlist tracks
playlist = sp.playlist(playlist_uri)
tracks = playlist['tracks']['items']

# Initialize data structures
song_features = []

# Define offset for pagination
offset = 0
limit = 100

# Iterate through playlist tracks, making additional requests as needed
while True:
    # Get playlist tracks with pagination
    results = sp.user_playlist_tracks(user="George Pacitto", playlist_id=playlist_uri, offset=offset, limit=limit)

    # Check if there are more tracks to retrieve
    if not results['items']:
        break

    # Extract track information and audio features
    for track in results['items']:
        track_uri = track['track']['uri']
        track_name = track['track']['name']

        # Extract track information
        track_info = sp.track(track_uri)
        artist_name = track_info['artists'][0]['name']
        track_name = track_info['name']
        track_id = track_info['id']
        album_name = track_info['album']['name']
        album_id = track_info['album']['id']
        release_date = track_info['album']['release_date']
        duration_ms = track_info['duration_ms']
        popularity = track_info['popularity']
        explicit = track_info['explicit']
        
        
        # Extract audio features
        track_features = sp.audio_features(track_uri)

        # Check if audio features are available
        if track_features:
            track_features = track_features[0]

            # Extract audio features
            danceability = track_features['danceability']
            energy = track_features['energy']
            key = track_features['key']
            loudness = track_features['loudness']
            mode = track_features['mode']
            speechiness = track_features['speechiness']
            acousticness = track_features['acousticness']
            instrumentalness = track_features['instrumentalness']
            liveness = track_features['liveness']
            valence = track_features['valence']            
            tempo = track_features['tempo']
            time_signature = track_features['time_signature']
            
            # get featured artists
            featured_artists = []
            if len(track_info['artists']) > 1:
                feat_artists = []
                for j in range(1, len(track_info['artists'])):
                    feat_artists.append(track_info['artists'][j]['name'])
                featured_artists.append(feat_artists)
            else:
                featured_artists.append([])
   

            # Store data
            song_features.append([
                artist_name, track_name, track_id, album_name,
                album_id, release_date, duration_ms, popularity,
                explicit, danceability, energy, key, loudness,
                mode, speechiness, acousticness, instrumentalness,
                liveness, valence, tempo, time_signature, featured_artists
            ])
        else:
            print("Audio features not available for track:", track_name)

    # Update offset for next request
    offset += limit

# Save data to CSV file
ft = pd.DataFrame(
    song_features, 
    columns=[ 'artist_name', 'track_name', 'track_id', 'album_name', 'album_id',
                'release_date', 'duration_ms', 'popularity', 'explicit',
                'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                'acousticness', 'instrumentalness', 'liveness',
                'valence', 'tempo', 'time_signature', 'featured_artists'])

# end time and print
end_time = timer()
elapsed_time = int(end_time - start_time)
print(f"Elapsed time:", timekeeper(elapsed_time))

Elapsed time: 1 minutes, 34 seconds


In [42]:
ft

,artist_name,track_name,track_id,album_name,album_id,release_date,duration_ms,popularity,explicit,danceability,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,8 Ounz,College Girls (feat. Kendrick Lamar),5MDvjyhIVrl7Lt8z0YEflo,"Hip Hop Had a Dream: The World Wide Tape, Vol. 2",6zBqDiLjTsqFdzYTusNDCn,2012-03-05,273737,11,True,0.788,...,-4.665,0,0.2430,0.24600,0.000000,0.7450,0.757,93.965,4,[[]]
1,Jay Rock,Hood Gone Love It (feat. Kendrick Lamar),6i6cWTCYBOKEdJnWjzm2yk,Follow Me Home,6bPbKuFNW6Vausf1PExvd9,2011-07-26,245466,67,True,0.636,...,-3.974,1,0.3120,0.08790,0.000000,0.2890,0.677,91.810,4,[[Kendrick Lamar]]
2,Lyn Charles,Star Life (feat. Kendrick Lamar),2ZblN6mtRfOB3sS8Z8bLY6,Star Life (feat. Kendrick Lamar),3acn5bSSEvwxhrsmti72fg,2012-03-04,209059,20,False,0.789,...,-1.871,0,0.2580,0.04510,0.000001,0.3140,0.412,122.152,5,[[]]
3,Birdman,B-Boyz,5MpIrad80ZF5rSPbovUfsZ,Kiss The Ring (Deluxe),5brUmg8Gox3t2OmAQzLivJ,2012-01-01,297160,0,True,0.491,...,-3.568,1,0.3010,0.02190,0.000000,0.0698,0.445,96.563,5,"[[Mack Maine, Kendrick Lamar, Ace Hood, DJ Kha..."
4,Talib Kweli,Push Thru,77eyE4gfO6iVtbD3CoqblG,Prisoner of Conscious,7kAg6DbRZXNLKzvQAdYkJp,2013-05-07,298255,0,True,0.372,...,-5.463,1,0.1700,0.06760,0.000002,0.0757,0.453,79.784,4,"[[Curren$y, Kendrick Lamar, Glen Reynolds]]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,BJ The Chicago Kid,His Pain,2uujJLrRplaQMkEx8CybZ8,Pineapple Now-Laters,2pGf7mMaV60uMDchGAMcXF,2012-02-21,329000,32,False,0.447,...,-11.076,0,0.1670,0.87200,0.000006,0.1020,0.770,165.695,4,[[]]
170,ScHoolboy Q,Overtime,5lsrKf4mzIO4gA9U9eBYms,Blank Face LP,0YbpATCIng8Fz2JrfHmEf7,2016-07-08,278493,42,True,0.813,...,-10.590,1,0.3320,0.77200,0.000000,0.1050,0.375,118.018,4,"[[Miguel, Justine Skye]]"
171,Baby Keem,range brothers (with Kendrick Lamar),7vy76jf6qE0IvoREJrvZtg,The Melodic Blue,7n23fjZTviIUnHyvZGQjni,2021-09-22,316733,44,True,0.663,...,-5.789,0,0.2750,0.00472,0.000000,0.1250,0.146,121.678,4,[[Kendrick Lamar]]
172,Beyoncé,AMERICA HAS A PROBLEM (feat. Kendrick Lamar),5xlLG6RthtyXShVhGA3ojN,AMERICA HAS A PROBLEM (feat. Kendrick Lamar),2WMGh4pKjsJ21Jlj1DzT7c,2023-05-19,260962,44,False,0.788,...,-5.308,1,0.0372,0.01100,0.000015,0.1420,0.229,126.042,4,[[Kendrick Lamar]]


Creating a 'type' field for the ft DataFrame and categorizing everything as a 'Feature' before concatenating it with the kdot DataFrame

In [77]:
ft['type']='Feature' 

df = pd.concat([kdot, ft], ignore_index= True, sort= False)

Eliminate Duplicates ie. rows with the same _track name_

In [78]:
df = df.sort_values(by= ['track_name', 'popularity'], ascending= [True, False])
df.drop_duplicates(subset= 'track_name', keep= 'first', inplace= True, ignore_index= True)
df

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists,type
0,Big Sean,"100 (feat. Royce da 5'9"", Kendrick Lamar)",6P4likikgh5T40Xf9jAfX9,Good Music Chicago,4TQQatT1Z2q4iIPcja4zGB,2013-02-19,11,False,0.400,336133,...,1,0.3340,0.33200,0.000000,0.6430,0.393,93.223,4,"[[Kendrick Lamar, Royce Da 5'9""]]",Feature
1,Rich Gang,100 Favors,4JSHYUTnIpDOGxaOwUEpXg,Rich Gang (Deluxe Version),7LSauzoqSHqCbmrYppgReX,2013-01-01,0,True,0.627,248106,...,1,0.0640,0.21700,0.000000,0.1110,0.120,74.018,4,"[[Detail, Birdman, Kendrick Lamar]]",Feature
2,A$AP Rocky,1Train,7AijU6oTPGmG64uWf63Qvc,LONG.LIVE.A$AP (Deluxe Version),1E1eyI5uGllppJZCxNoF9w,2013-01-11,0,True,0.622,372173,...,1,0.3320,0.34900,0.000000,0.6950,0.768,83.568,4,"[[Kendrick Lamar, Joey Bada$$, Yelawolf, Danny...",Feature
3,Kendrick Lamar,A.D.H.D,2Fw5S2gaOSZzdN5dFoC2dj,Section.80,1bkN9nIkkCnXeG4yitVS1J,2011-07-02,72,True,0.609,215509,...,1,0.2400,0.33000,0.000000,0.1080,0.582,157.740,4,[],Studio Album
4,Meek Mill,A1 Everything (feat. Kendrick Lamar),5Kj6eCkv21z5TPEh3Z6POl,Dream Chaser 4,1uLT8uuhGWdJI82RJ1AkV9,2015-06-15,0,True,0.698,173440,...,1,0.3750,0.00698,0.000000,0.3330,0.840,156.135,4,[[Kendrick Lamar]],Feature
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,Kendrick Lamar,untitled 06 | 06.30.2014.,4M2t7bP4Mq87mGMn0PObUX,untitled unmastered.,0kL3TYRsSXnu0iJvFO3rud,2016-03-04,57,True,0.669,208013,...,0,0.0461,0.09920,0.000008,0.2540,0.645,89.083,4,[],Compilation
240,Kendrick Lamar,untitled 07 | 2014 - 2016,40IQooNkPRVtn4zlE3ZFpA,untitled unmastered.,0kL3TYRsSXnu0iJvFO3rud,2016-03-04,54,True,0.567,496173,...,0,0.5040,0.73200,0.000000,0.7090,0.324,126.982,4,[],Compilation
241,Kendrick Lamar,untitled 07 | levitate,6QvxTX9zvo4u7XhP9vOP3n,untitled 07 | levitate,1EmfbMC739Pfi2Q3U4a0Y7,2016-03-23,50,True,0.790,146467,...,1,0.2230,0.49000,0.000000,0.0884,0.530,126.992,4,[],Single
242,Kendrick Lamar,untitled 08 | 09.06.2014.,5bBUDJUfGcG7eFy3Bf4fXv,untitled unmastered.,0kL3TYRsSXnu0iJvFO3rud,2016-03-04,65,True,0.850,235507,...,0,0.2040,0.25000,0.000002,0.1140,0.686,120.004,4,[],Compilation


Drop rows from the _type_ field with unassigned values

In [79]:
notype = df[df['type'] =='']
notype[['artist_name', 'track_name', 'album_name', 'type']]

,artist_name,track_name,album_name,type
85,Kendrick Lamar,HOC,We Run L.A.,
86,Kendrick Lamar,HUMBLE.,2018 Award Nominees,
113,Kendrick Lamar,LOVE.,Top 30 US,
208,Kendrick Lamar,War Is My Love,Gangsta Bangaz Vol. 1,
225,Kendrick Lamar,You Don't Like Me,"Beaches & Bridges - La 2 da Bay, Vol. 5",


In [81]:
notype = df[df['type'] =='']
flush = notype.index
df = df.drop(flush)
notype[['artist_name', 'track_name', 'album_name', 'type']]

,artist_name,track_name,album_name,type


Modifying the 'mode' column by replacing 0 with Minor and 1 with Major

In [84]:
df['mode'] = df['mode'].replace({0:'Minor', 1:'Major'})
df[['track_name', 'album_name', 'mode']].tail()

,track_name,album_name,mode
239,untitled 06 | 06.30.2014.,untitled unmastered.,Minor
240,untitled 07 | 2014 - 2016,untitled unmastered.,Minor
241,untitled 07 | levitate,untitled 07 | levitate,Major
242,untitled 08 | 09.06.2014.,untitled unmastered.,Minor
243,vent,The Melodic Blue,Major


Normalizing the _duration ms_ column to appear as minutes and seconds 

In [85]:
df['duration_ms'] = pd.to_timedelta(df['duration_ms'], unit='ms')

# extract the minutes and seconds components of the duration as strings
df['duration'] = df['duration_ms'].dt.components['minutes'].astype(str).str.zfill(2) + ':' + df['duration_ms'].dt.components['seconds'].astype(str).str.zfill(2)
df.drop(columns=['duration_ms'], inplace= True)
df[['track_name','duration']].tail()

,track_name,duration
239,untitled 06 | 06.30.2014.,03:28
240,untitled 07 | 2014 - 2016,08:16
241,untitled 07 | levitate,02:26
242,untitled 08 | 09.06.2014.,03:55
243,vent,02:16


Creating three new fields: year, month and day a song was released from release date column

In [ ]:
...